In [1]:
import tensorflow as tf
import tensorflow_gnn as tfgnn

ModuleNotFoundError: No module named 'tensorflow_gnn'

In [ ]:
#model hyperparameters
h_dims={'user':256,'movie':64,'genre':128}

In [ ]:
#model builder initialization
model_builder=tfgnn.keras.ConvGNNBuilder(
    lambda edge_set_name: WeightedSumConvolution()
    lambda node_set_name: tf.keras.layers.NextStateFromConcat(
        tf.keras.layers.Dense(h_dims[node_set_name])
    )
)

In [ ]:
#two rounds of message passing to target node sets
model=tf.keras.models.Sequential([
    gnn.Convolve({'genre'}),#sends messages from 'movie' to 'genre'
    gnn.Convolve({'user'})#sends messages from movie and genre to 'user'
    tfgnn.keras.layers.Readout(node_set_name='user'),
    tf.keras.layers.Dense(1)
])

In [ ]:
# tensorflow dataset movie_lens_100k
import tensorflow_datasets as tfds

# load the dataset
movielens, movielens_info = tfds.load('movielens/100k-ratings', split='train', with_info=True)
print(movielens_info.description)

In [ ]:
class WeightedSumConvolution(tf.keras.layers.Layer):
    # weight sum of source node states
    def call(self,graph:tfgnn.GraphTensor,edge_set_name:tfgnn.EdgeSetName ) -> tfgnn.Field:
        messages=tfgnn.broadcast_node_to_edges(
            graph,
            edge_set_name,
            tfgnn.SOURCE,
            feature_name=tfgnn.DEFAULT_SET_NAME)
        weights=  graph.edge_sets[edge_set_name]['weight']
        weighted_messages=messages*tf.expand_dims(weights,-1)
        pooled_messages=tfgnn.pool_edges_to_nodes(
            graph,
            edge_set_name,
            tfgnn.TARGET,
            reduce_type='sum',
            feature_value=weighted_messages)
        return pooled_messages
        




In [2]:
# tensorflow dataset movie_lens_100k
import tensorflow_datasets as tfds

# load the dataset
movielens, movielens_info = tfds.load('movielens/100k-ratings', split='train', with_info=True)
print(movielens_info.description)

C:\Users\Navneetkrh\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This dataset contains a set of movie ratings from the MovieLens website, a movie
recommendation service. This dataset was collected and maintained by [GroupLens]
(https://grouplens.org/), a research group at the University of Minnesota. There
are 5 versions included: "25m", "latest-small", "100k", "1m", "20m". In all
datasets, the movies data and ratings data are joined on "movieId". The 25m
dataset, latest-small dataset, and 20m dataset contain only movie data and
rating data. The 1m dataset and 100k dataset contain demographic data in
addition to movie and rating data.

- "25m": This is the latest stable version of the MovieLens dataset. It is
recommended for research purposes.
- "latest-small": This is a small subset of the latest version of the MovieLens
dataset. It is changed and updated over time by GroupLens.
- "100k": This is the oldest version of the MovieLens datasets. It is a small
dataset with demographic data.
- "1m": This is the largest MovieLens dataset that contains dem

In [ ]:
movielens_info

In [5]:
# prepare the dataset
def prepare_movielens_dataset(raw_movielens):
    # extract the features
    user_id=raw_movielens['user_id']
    movie_id=raw_movielens['movie_id']
    rating=raw_movielens['rating']
    genre=raw_movielens['movie_genres']
    # convert the features to tensors
    user_id=tf.cast(user_id,tf.int32)
    movie_id=tf.cast(movie_id,tf.int32)
    rating=tf.cast(rating,tf.float32)
    genre=tf.cast(genre,tf.float32)
    # build the graph
    graph=tfgnn.GraphTensor(
        nodes={
            'user':user_id,
            'movie':movie_id,
            'genre':genre
        },
        edges={
            'watched':tfgnn.EdgeSet(
                source='user',
                target='movie',
                node_ids=tf.range(tf.size(user_id)),
                weight=tf.ones_like(user_id,dtype=tf.float32)
            ),
            'has_genre':tfgnn.EdgeSet(
                source='movie',
                target='genre',
                node_ids=tf.range(tf.size(movie_id)),
                weight=tf.ones_like(movie_id,dtype=tf.float32)
            )
        }
    )
    return graph,rating

In [6]:
    #  prepare the dataset
    movielens_prep=movielens.map(prepare_movielens_dataset)

NameError: in user code:

    File "C:\Users\Navneetkrh\AppData\Local\Temp\ipykernel_21088\2387456226.py", line 14, in prepare_movielens_dataset  *
        graph=tfgnn.GraphTensor(

    NameError: name 'tfgnn' is not defined
